## More `bysort`, `merge` and multivariate regression using Stata

In [1]:
qui cd Z:/ECON-C4100 // change working dir
qui use data/sweden_prices.dta, clear

### Calculating unique observation within groups using bysort

We start with `bysort` again. This time I demonstrate how you can use it to calculate the number of observations or products within some subgroups and variables. Quite often we are interested in calculating some group specific statistics such as sums, means and covariances (see the materials for Session 3). However, quite often we want to calculate the number of observations or more specifically, the number of **unique** or **distinct** observations.

Let's demonstrate. We want to calculate how many products each company or `Företag` has on the market.

In [2]:
qui {
    bysort Företag Produktnamn: gen n_products1 = _n == 1
    bysort Företag: replace n_products1 = sum(n_products1)
    bysort Företag: replace n_products1 = n_products1[_N]
}
tab n_products1 if strpos(Företag, "Orion") > 0



. tab n_products1 if strpos(Företag, "Orion") > 0

n_products1 |      Freq.     Percent        Cum.
------------+-----------------------------------
        132 |        515      100.00      100.00
------------+-----------------------------------
      Total |        515      100.00


The result is shown above. Orion has 131 products at the market, separated by the product name.

The above code does the following things:

1. Bysort "creates" subgroups for `Företag` and `Produktnamn` with their own sort orders
2. Tag the first value of each subgroup, ie. `_n == 1`
3. Calculate the (rolling) sum of tags by `Företag` with `sum`.
4. The total is the value of the last value of `Företag`, ie. `_N`.

I call them them "the Three Lines of God-tier Code" because I use them so often. [Kudos to Nick Cox again!](https://www.stata.com/support/faqs/data-management/number-of-distinct-observations/)

It's a nice trick to know and master even for cross-section data. But it's even better for panel data. The basic pricinple remains the same and can be stated as:

```
bysort timevar xvar zvar: gen nvals = _n == 1
by timevar xvar: replace nvals = sum(nvals)
by timevar xvar: replace nvals = nvals[_N]

```

Which calculates the number of distinct observations of `zvar` within `timevar` and `xvar`.


However, there is also another way to do the same thing with `egen`:

In [3]:
qui {
    egen n_products2_temp = tag(Företag Produktnamn)
    bysort Företag: egen n_products2 = total(n_products2_temp)
    drop n_products2_temp
}
tab n_products2 if strpos(Företag, "Orion") > 0



. tab n_products2 if strpos(Företag, "Orion") > 0

n_products2 |      Freq.     Percent        Cum.
------------+-----------------------------------
        132 |        515      100.00      100.00
------------+-----------------------------------
      Total |        515      100.00


The answer is the same.

### Calculating imaginary sales data using `bysort`

`bysort` is the go-to tool to calculate other stats too. Here I demonstrate how you can calculate company sales and market shares with imaginary simulated sales volume.

It's an ugly demonstration but works for now.

In [4]:
qui {
    gen sales_volume =  runiform(1,5000)
    bysort ATCkod: egen total_sales = total(sales_volume)
    bysort ATCkod Företag: egen firm_sales = total(sales_volume)
    gen market_share = (firm_sales/total_sales)*100
}
tab market_share if strpos(Företag, "Orion") > 0 & strpos(ATCkod, "C10AA01") > 0



. tab market_share if strpos(Företag, "Orion") > 0 & strpos(ATCkod, "C10AA01") > 0

market_shar |
          e |      Freq.     Percent        Cum.
------------+-----------------------------------
   13.70447 |          7      100.00      100.00
------------+-----------------------------------
      Total |          7      100.00


### Duplicate observations

Duplicate observations are observations that have identical values in their variables. A natural way to calculate the number of duplicate observations by some variable list can be done using the `duplicates` command(s):

In [5]:
duplicates report _all


Duplicates in terms of Produktnamn Varunummer ATCkod NPLid NPLpackid Form Styrka Förpackning Antal Företag AIP AUP AIPperst AUPperst Subventionerad dummy atc_1 cond_mean cond_mean_2 cond_mean_3 cond_mean_4 name_length correlation covariance n_products1
    n_products2 sales_volume total_sales firm_sales market_share

--------------------------------------
   copies | observations       surplus
----------+---------------------------
        1 |        13953             0
--------------------------------------


There are no perfect duplicates in our Swedish data. What about VNRs?

In [6]:
duplicates report Varunummer


Duplicates in terms of Varunummer

--------------------------------------
   copies | observations       surplus
----------+---------------------------
        1 |        12770             0
        2 |          514           257
        3 |          279           186
        4 |          144           108
        5 |           40            32
        6 |           78            65
        7 |           56            48
        8 |           24            21
        9 |           18            16
       10 |           30            27
--------------------------------------


Some observations have the same VNR but different values for some of the other variables. Why?

The answer: the cross-section variable in the Swedish data is not the Nordic article number (VNR) but `NPLpackid`. Let's try:

In [8]:
duplicates report NPLpackid


Duplicates in terms of NPLpackid

--------------------------------------
   copies | observations       surplus
----------+---------------------------
        1 |        13953             0
--------------------------------------


No duplicates. But we can create some using the `expand` command that creates duplicates from existing observations:

In [9]:
expand 2
duplicates report NPLpackid


(13,953 observations created)


Duplicates in terms of NPLpackid

--------------------------------------
   copies | observations       surplus
----------+---------------------------
        2 |        27906         13953
--------------------------------------


The `expand` command is extremely useful when dealing with panel and time series data. But let's get back to the `duplicates` command. Or to be more specific, I'll demonstrate the `duplicates tag` and `duplicates drop` command:

In [11]:
duplicates tag NPLpackid, gen(tag)
tab tag
drop tag
duplicates drop NPLpackid, force
duplicates report NPLpackid



Duplicates in terms of NPLpackid


        tag |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |     13,953      100.00      100.00
------------+-----------------------------------
      Total |     13,953      100.00



Duplicates in terms of NPLpackid

(0 observations are duplicates)


Duplicates in terms of NPLpackid

--------------------------------------
   copies | observations       surplus
----------+---------------------------
        1 |        13953             0
--------------------------------------


`duplicates tag` saves the number of surplus duplicates to a new variabe specified by the user. The `duplicates drop` command drops the duplicate observations (it also requires the option `force`).

```{warning}

You should not use `duplicates drop` without if you do not know which observations you are dealing with!

```

### Combining data sets - introduction to `merge`

Quite often we want to use data from multiple source. How can we do that? We need at least:

1. Some new data related to our current data
2. key(s) to link you current data with the new data
3. some command to do this for us

Parts 1 and 2 are situational. But Stata can help us with the part 3 with the `merge` command. It is used to add new existing variables to our data.

![](figures/merge.png)

The data we want to add is the "product of the month" substitution data from Sweden.

```{note}

You may want to update our Swedish price data from Session 1. You can do this by importing it again and using `save` with `replace`.

```

We will other data combining functions more in detail in the next course.

Let's clear the current memory and import it to Stata:

In [12]:
clear all
import excel using ///
"https://www.tlv.se/webdav/files/Periodens-vara/periodens-vara-februari-20210201.xlsx", ///
firstrow clear // variable names in first row
describe





Contains data
  obs:         6,683                          
 vars:            17                          
 size:     2,459,344                          
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Status          str3    %9s                   Status
Produktnamn     str50   %50s                  Produktnamn
Varunummer      str6    %9s                   Varunummer
Styrka          str36   %36s                  Styrka
Förpac

Let's keep only `NPLpackID`, `UtbytesgruppsID` and `Status`.

In [13]:
keep NPLpackID UtbytesgruppsID Status

Our key for combining data is `NPLpackID`. For the `merge` to work, the variable names in our two data sets have to be equal. We'll have to `rename`.

In [14]:
rename NPLpackID NPLpackid

Now we can `merge`. There are three variants of merge:

0. 1 to 1
1. 1 to many
2. many to 1 
3. many to many

The first one means that there are more than 1 observations for our key in the other data set. The second means that we have duplicates for our key in this data set but there is only one value in the other data set. The many-to-many merge exists, but the Stata manual says that you should never use it.

We have 1:1 merge because we are using the `NPLpackid`.

In [15]:
merge 1:1 NPLpackid using "Z:/ECON-C4100/data/sweden_prices.dta"


    Result                           # of obs.
    -----------------------------------------
    not matched                         7,270
        from master                         0  (_merge==1)
        from using                      7,270  (_merge==2)

    matched                             6,683  (_merge==3)
    -----------------------------------------


The output tells us that slightly less than half of the observations were matched. The information whether some observation was matched is saved in the variable `_merge` by default.

With our new data, we can `generate` a new variable for the substitution status. This time we will call it the "product of the month status":

In [16]:
gen pm_status = _merge == 3
tab pm_status




  pm_status |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |      7,270       52.10       52.10
          1 |      6,683       47.90      100.00
------------+-----------------------------------
      Total |     13,953      100.00


Let's also add a dummy for each ATC level 1 in the data:

In [17]:
egen atc1_group = group(atc_1), label
tab atc_1, gen(atc_dummy)




      atc_1 |      Freq.     Percent        Cum.
------------+-----------------------------------
          A |      1,408       10.09       10.09
          B |        956        6.85       16.94
          C |      1,429       10.24       27.18
          D |        412        2.95       30.14
          G |        544        3.90       34.04
          H |        400        2.87       36.90
          J |      1,032        7.40       44.30
          L |      1,754       12.57       56.87
          M |        403        2.89       59.76
          N |      4,408       31.59       91.35
          P |         45        0.32       91.67
          R |        691        4.95       96.62
          S |        349        2.50       99.13
          V |        122        0.87      100.00
------------+-----------------------------------
      Total |     13,953      100.00


In [20]:
tab atc1_group
describe atc_dummy*



group(atc_1 |
          ) |      Freq.     Percent        Cum.
------------+-----------------------------------
          A |      1,408       10.09       10.09
          B |        956        6.85       16.94
          C |      1,429       10.24       27.18
          D |        412        2.95       30.14
          G |        544        3.90       34.04
          H |        400        2.87       36.90
          J |      1,032        7.40       44.30
          L |      1,754       12.57       56.87
          M |        403        2.89       59.76
          N |      4,408       31.59       91.35
          P |         45        0.32       91.67
          R |        691        4.95       96.62
          S |        349        2.50       99.13
          V |        122        0.87      100.00
------------+-----------------------------------
      Total |     13,953      100.00


              storage   display    value
variable name   type    format     label      variable label
----------

### Multiple regressions using Stata

Doing multiple regressions using Stata is as easy as univariate regressions. We simply add more variables to the equation. Let's continue with our light-hearted example from the previous session:

In [21]:
reg AIP name_length pm_status atc_dummy*, r

note: atc_dummy11 omitted because of collinearity

Linear regression                               Number of obs     =     13,953
                                                F(15, 13937)      =      67.36
                                                Prob > F          =     0.0000
                                                R-squared         =     0.1229
                                                Root MSE          =      10160

------------------------------------------------------------------------------
             |               Robust
         AIP |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
 name_length |  -176.9473   10.54853   -16.77   0.000    -197.6239   -156.2708
   pm_status |  -1862.504    171.648   -10.85   0.000    -2198.957   -1526.051
  atc_dummy1 |     632.32    316.969     1.99   0.046     11.01831    1253.622
  atc_dummy2 |   4003.285   588.1725     6.

We could have also typed:

In [22]:
reg AIP name_length pm_status i.atc1_group, r


Linear regression                               Number of obs     =     13,953
                                                F(15, 13937)      =      67.36
                                                Prob > F          =     0.0000
                                                R-squared         =     0.1229
                                                Root MSE          =      10160

------------------------------------------------------------------------------
             |               Robust
         AIP |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
 name_length |  -176.9473   10.54853   -16.77   0.000    -197.6239   -156.2708
   pm_status |  -1862.504    171.648   -10.85   0.000    -2198.957   -1526.051
             |
  atc1_group |
          B  |   3370.965   573.2447     5.88   0.000     2247.329    4494.602
          C  |   887.6416   230.0974     3.86   0.000     436.6

There are a few important things to notive. First of all, `i.` in front of some variable means that we want Stata to consider this variable as a **categorical** variable. Stata will automatically add the dummy variables for us. Remember that one dummy goes always to zero because of multicollinearity.

Second of all, we've used the option `r` in our regressions. This tells Stata to calculate robust standard errors that allow for heteroscedasticity. We will use this option in all of our estimations in the future and so should you.

Finally, let's transform some of the variables to natural logs and run our regression again:

In [23]:
gen ln_nlength = ln(name_length)
gen ln_AIP = ln(AIP)
reg ln_AIP ln_nlength pm_status i.atc1_group, r





Linear regression                               Number of obs     =     13,953
                                                F(15, 13937)      =     561.18
                                                Prob > F          =     0.0000
                                                R-squared         =     0.3903
                                                Root MSE          =       1.48

------------------------------------------------------------------------------
             |               Robust
      ln_AIP |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
  ln_nlength |  -.6799245   .0298059   -22.81   0.000     -.738348    -.621501
   pm_status |    -.99519   .0297118   -33.49   0.000    -1.053429   -.9369509
             |
  atc1_group |
          B  |   1.380389   .0603043    22.89   0.000     1.262185    1.498594
          C  |  -.0759281   .0557473    -1.36   0.173    -.1

### Save data for later use

In [24]:
save data/sweden_prices_merged.dta, replace

file data/sweden_prices_merged.dta saved
